1. TRACE: The TRACE Level designates finer-grained informational events than the DEBUG.
2. DEBUG: The DEBUG Level designates fine-grained informational events that are most useful to debug an application.
3. INFO: The INFO level designates informational messages that highlight the progress of the application at coarse-grained level.
4. WARN: The WARN level designates potentially harmful situations.
5. ERROR: The ERROR level designates error events that might still allow the application to continue running.
6. FATAL: The FATAL level designates very severe error events that will presumably lead the application to abort.
7. ALL: The ALL has the lowest possible rank and is intended to turn on all logging.
8. OFF: The OFF has the highest possible rank and is intended to turn off logging.

Rank\Priority order: `ALL < TRACE < DEBUG < INFO < WARN < ERROR < FATAL < OFF`

If we set level to `WARN` then only `warn`,  `ERROR`, and `FATAL` gets printed.

```
Synapse Spark configures default level to `INFO` so trace and debug can't be printed.
```

## Demonstrace default behavior

By default, the following are logged to Log Analytics:

- INFO
- WARN 
- ERROR
- FATAL

### Log4j 1.x

In [1]:
//Log4j 1.x API
val log = org.apache.log4j.LogManager.getLogger(s"com.aravind.notebook.log4j.1x")

//sc.setLogLevel("DEBUG")
//Rank order ALL < TRACE < DEBUG < INFO < WARN < ERROR < FATAL < OFF

log.trace("log4j.1x TRACE")
log.debug("log4j.1x DEBUG")
log.info("log4j.1x INFO")
log.warn("log4j.1x WARN")
log.error("log4j.1x ERROR")
log.fatal("log4j.1x FATAL")

In [2]:
import com.tccc.dna.synapse.Logs._
import com.tccc.dna.synapse.spark.SynapseSpark

val notebookName = SynapseSpark.getCurrentNotebookName
val aravindLog = org.apache.log4j.LogManager.getLogger(s"com.aravind.notebook.$notebookName.log4j.1x")
//aravindLog.setLevel(org.apache.log4j.Level.DEBUG)
logTrace(aravindLog, "Logs api - Trace message")
logDebug(aravindLog, "Logs api - Debug message")
logInfo(aravindLog, "Logs api - Info message")
logWarning(aravindLog, "Logs api - Warn message")
logError(aravindLog, "Logs api - Error message")
logFatal(aravindLog, "Logs api - Fatal message")

### Log4j 2.x - Using LogManager

`LogManager` is a static utility class that manages all loggers within an application. When you call this method, it checks if the requested logger already exists; if not, it creates a new one based on the log4j2 configuration.

In [3]:
//Log4j 2.x API
import org.apache.logging.log4j.core._

val rootLogger = org.apache.logging.log4j.LogManager.getLogger(s"com.aravind.notebook.log4j.2x")

rootLogger.trace("log4j.2x root TRACE")
rootLogger.debug("log4j.2x root DEBUG")
rootLogger.info("log4j.2x root INFO")
rootLogger.warn("log4j.2x root WARN")
rootLogger.error("log4j.2x root ERROR")
rootLogger.fatal("log4j.2x root FATAL")
println(rootLogger.isTraceEnabled)
println(rootLogger.isDebugEnabled)
println(rootLogger.isInfoEnabled)

### Log4j 2.x - Using LoggerContext

Use `LoggerContext` when you want to manipulate the logging configuration programmatically or load a custom configuration. In general, you will likely use `LogManager` more often since it's the primary entry point for logging in most applications. LoggerContext is used when you need advanced control over Log4j configurations.

In [4]:
val cctx = LoggerContext.getContext()

println(cctx.getName)

val log2=cctx.getLogger("aravind.LoggerContext API")
log2.trace("log2 TRACE")
log2.debug("log2 DEBUG")
log2.info("log2 INFO")
log2.warn("log2 WARN")
log2.error("log2 ERROR")
log2.fatal("log2 FATAL")

## Exceptions

In [15]:
log.error("Exception without message", new RuntimeException)
log.error("Exception with message", new RuntimeException("Something went wrong"))
log.fatal("Exception without message", new IllegalStateException)
log.fatal("Exception with message", new IllegalStateException("Something went wrong"))

In [16]:
try {
    throw  new NullPointerException("Test stack trace")
} catch  {
    case ex: Exception => {
        log.error("Exception Caguht nullpointer in try/catch", ex)
        log.fatal("Exception Caguht nullpointer in try/catch", ex)
        logError(aravindLog, "Exception Caguht nullpointer in try/catch", ex)
        rootLogger.error("Exception Caguht nullpointer in try/catch", ex)
        rootLogger.fatal("Exception Caguht nullpointer in try/catch", ex)
        //log2.fatal("Exception Caguht nullpointer in try/catch", ex)
        //log2.error("Exception Caguht nullpointer in try/catch", ex)
        //log2.fatal("Exception Caguht nullpointer in try/catch", ex)
    }
}

### Exception in Worker

In [17]:
import org.apache.spark.sql.Row
import org.apache.spark.sql.types._

val numRdd = sc.parallelize(Seq(
    Row(1,1), Row(1,0), Row(2,0)
))

val schema = StructType(Array(
    StructField("a",DoubleType,true),
    StructField("b",DoubleType,true)
    ))

val numDf = spark.createDataFrame(numRdd, schema)
numDf.createOrReplaceTempView("number_tbl")

## Open issue with product team: 

When workers raise exceptions, the following columns are not populated. We expect these to be populated. These are populated when the exception is raised from driver though.
exception_exception_class_s
exception_exception_message_s
exception_stacktrace_

In [18]:
%%sql
SELECT a, b from number_tbl

##  Disable Log Level

In [19]:
println(log)
println(aravindLog)
//println(log2)

sc.setLogLevel("ERROR")

log.debug("after changing log levl to ERROR - DEBUG msg")
log.info("after changing log levl to ERROR - INFO msg")
log.warn("after changing log levl to ERROR - WARN msg")
log.error("after changing log levl to ERROR - ERROR msg")
log.fatal("after changing log levl to ERROR - FATAL msg")

aravindLog.info("after changing log levl to ERROR - DEBUG msg")
aravindLog.info("after changing log levl to ERROR - INFO msg")
aravindLog.info("after changing log levl to ERROR - WARN msg")
aravindLog.error("after changing log levl to ERROR - ERROR msg")
aravindLog.fatal("after changing log levl to ERROR - FATAL msg")

//Change it back
sc.setLogLevel("INFO")
aravindLog.info("after changing log levl back to INFO - INFO msg")
aravindLog.info("after changing log levl back to INFO - WARN msg")

**The root logger is set to `INFO`. We can change it to to `DEBUG`** as shown below but it is highly recommended to use this only if absolutely necessary. 

NOTE: Always log at INFO level.

In [23]:
import org.apache.logging.log4j.LogManager
import org.apache.logging.log4j.core.LoggerContext
val ctx = LogManager.getContext(false).asInstanceOf[LoggerContext]
val config = ctx.getConfiguration
config.removeFilter(config.getFilter)


sc.setLogLevel("DEBUG")
log.debug("after changing log level to DEBUG - DEBUG msg")
log.info("after changing log level to DEBUG - INFO msg")

sc.setLogLevel("INFO")
log.debug("after changing log level to INFO - DEBUG msg")
log.info("after changing log level to INFO - INFO msg")

## MDC

In [10]:
import org.apache.log4j.MDC
MDC.put("TestKey 1", "TestValue 1")
MDC.put("TestKey 2", "TestValue 2")

spark.sparkContext.setLocalProperty("mdc." + "name", "value")


log.info("Test MDC")

In [14]:
val driverLog4jPath = System.getProperty("log4j2.configurationFile")
val rootLogLevel = System.getProperty("log4jspark.root.logger").split(",")(0)
val log4jAppenders = System.getProperty("log4jspark.root.logger").split(",").slice(1,2)
val logDirPath = System.getProperty("log4jspark.log.dir")
val user = System.getProperty("user.name")
import scala.sys.process._
//"hdfs dfs -copyFromLocal " + driverLog4jPath + " /"!!
//"hdfs dfs -copyFromLocal " + log4jPatternPath + " /"!!

In [13]:
val submitTime = System.getProperty("spark.app.submitTime")
val startTime = System.getProperty("spark.app.startTime")